In [1]:
import arviz as az
import pymc3 as pm
import pandas as pd
import pyro
import torch
import random
#from rethinking import MAP, coef, extract_samples, link, precis, sim, vcov
from pyro.contrib.autoguide import AutoLaplaceApproximation
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

adam_params = {"lr": 0.005, "betas": (0.95, 0.999)}

from pyro.optim import Adam

adam_params = {"lr": 0.005, "betas": (0.95, 0.999)}
optimizer = Adam(adam_params)
pyro.set_rng_seed(101)

In [26]:
#%load_ext watermark
%watermark -n -u -v -iv -w -a sushmit86@gmail.com

Author: sushmit86@gmail.com

Last updated: Mon Jan 18 2021

Python implementation: CPython
Python version       : 3.8.2
IPython version      : 7.16.1

pymc3    : 3.8
pandas   : 1.0.5
arviz    : 0.8.3
watermark: 2.1.0
torch    : 1.7.1
pyro     : 1.5.1

Watermark: 2.1.0



In [4]:
url = 'https://raw.githubusercontent.com/pymc-devs/resources/master/Rethinking/Data/Howell1.csv'
d = pd.read_csv(url,sep=';', header=0)
display(d.head(5))
d2 = d[d.age >= 18]

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041915,41.0,1
4,145.415,41.276872,51.0,0


In [39]:
# %timeit
# with pm.Model() as m4_1:
#   mu_model = pm.Normal('mu_model',mu = 178, sigma = 20)
#   sigma_model = pm.Uniform('sigma_model',0,50)
#   height = pm.Normal('height',mu = mu_model,sigma = sigma_model, observed = d2.height.values)

# with m4_1:
#   trace_4_1 = pm.sample(1000,tune=1000)

In [40]:
def flist(height):
    mu = pyro.sample("mu",  pyro.distributions.Normal(178, 20))
    sigma = pyro.sample("sigma",  pyro.distributions.Uniform(0, 50))
    with pyro.plate("plate"):
        pyro.sample("height",  pyro.distributions.Normal(mu, sigma), obs=height)

In [44]:
m4_1 = AutoLaplaceApproximation(flist)
svi = SVI(flist, m4_1, optimizer, Trace_ELBO(),d2.height.values)
n_steps = 5000
# do gradient steps
for step in range(n_steps):
    svi.step(torch.from_numpy(d2.height.values))